In [6]:
import vertexai # to interact with googles code chatbot ai
from vertexai.preview.language_models import CodeChatModel, ChatModel, InputOutputTextPair, ChatMessage,CodeGenerationModel, TextGenerationModel
import google
import openai
import openai.error

import syfco # to convert between different form
import bosy
from bosy import BosyError
import strix

import benchmark
from benchmark import Benchmark, build_prompt

import verify # used to verify verilog solutions
import json
import os

# both for handling async stuff
import asyncio

from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

import csv # to write the benchmark results to a csv file
from utils import *
from prompting import *
import time
import random

In [7]:
def generate_module_signature(bm: Benchmark, params=None, module_name="fsm"):
    if params == None:
        params = bm.generate_params
    spec = read_file(bm.specification)
    inputs = syfco.inputs(spec, overwrite_params=params)
    outputs = syfco.outputs(spec, overwrite_params=params)
    return f"""module {module_name} ({
    ", ".join(
      ["input " + inp for inp in inputs] + ["output reg " + out for out in outputs]
    )
  })"""


In [8]:
vertexai.init(project="rg-finkbeiner-30141001", location="us-central1")
chat_model = ChatModel.from_pretrained("chat-bison")
code_model = CodeChatModel.from_pretrained("codechat-bison")
parameters = {
    "temperature": 0.5,
    "max_output_tokens": 1024
}
openai.api_key = os.getenv("OPENAI_KEY")
openai.organization = os.getenv("OPENAI_ORG")
def openai_complete(messages, model = "gpt-3.5-turbo-16k", retry_n = 50, **kwargs):
    count = 0
    while count < retry_n:
        try:
            completion = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            **kwargs
            )
            return completion
        except (openai.error.RateLimitError,
                openai.error.APIError,
                openai.error.APIConnectionError,
                openai.error.ServiceUnavailableError,
                openai.error.Timeout,
                openai.error.TryAgain) as e: # OpenAI API seems to be quite unstable
            print(e)
            time.sleep(100)
            count = count + 1
    raise openai.error.RateLimitError

In [9]:
benchmarks = Benchmark.load_from_json("./benchmarks.json", "./parametric_solutions")
benchmarks_strix = Benchmark.load_from_json("./benchmarks_strix.json", "./syntcomp/tlsf")
benchmarks_bosy = Benchmark.load_from_json("./benchmarks_bosy.json", "./syntcomp/tlsf")

In [10]:
def improve_iteratively(response, bm):
  code = extract_normalized_verilog_code(response, bm.name)
  res = "NO_CODE" if code == None else bm.verify(impl=code).name
  match res:
    case "NO_CODE":
      prompt = "This doesn't seem like a complete verilog module. Could you please give me the entire module?"
    case "ERROR_CONVERT_TO_VERILOG":
      prompt = "This code isn't syntactically valid verilog code. Could you please fix the syntax errors?"
    case "ERROR_COMBINE_AIGER":
      prompt = f"Could you please make sure to use the module signature `{generate_module_signature(bm)}` in your code. Please also try to think about your code again using the new signature. Make sure it matches the specification!"
    case "FALSE_RESULT":
      prompt = "The code doesn't satisfy the specification. Please think extensively about how you need to change your code to satisfy the specification. If necessary, rewrite the module from ground up"
    case "SUCCESS":
      return (code, res)
    case _:
      return (None, res)
  return (prompt, res)

def run_single_iterative(bm, type):
  # bm.build_prompt might do some heavy work like synthesizing examples with bosy
  prompt, examples = build_prompt(bm, params=bm.generate_params, template=PromptPALM, mode=type)
  chat = chat_model.start_chat(examples=examples)
  response = chat.send_message(prompt, **parameters)
  for i in range(0, 3):
    (prompt, res) = improve_iteratively(response.text, bm)
    if res == "SUCCESS" or res == None:
      return res
    response = chat.send_message(prompt, **parameters)
  return res

def run_single_explicit_examples(bm, type):
    print("Starting benchmark " + bm.name + "/" + type)
    # bm.build_prompt might do some heavy work like synthesizing examples with bosy
    prompt, examples = build_prompt(bm, params=bm.generate_params, template=PromptPALM, mode=type)
    print("Finished generating prompt for " + bm.name + "/" + type)
    chat = chat_model.start_chat(examples=examples)
    response = chat.send_message(prompt, **parameters)
    code = extract_normalized_verilog_code(response.text, bm.name)
    # print(code if code else "NO_CODE::\n" + response.text)
    if code == None:
      return "NO_CODE"
    else:
      res = bm.verify(impl=code)
      return res.name

def run_single_explicit_examples_oneshot(bm, type):
    print("Starting benchmark " + bm.name + "/" + type)
    # bm.build_prompt might do some heavy work like synthesizing examples with bosy
    prompt, examples = build_prompt(bm, params=bm.generate_params, template=PromptPALM, mode=type, max_examples=1)
    print("Finished generating prompt for " + bm.name + "/" + type)
    print(len(examples))
    chat = chat_model.start_chat(examples=examples)
    response = chat.send_message(prompt, **parameters)
    code = extract_normalized_verilog_code(response.text, bm.name)
    # print(code if code else "NO_CODE::\n" + response.text)
    if code == None:
      return "NO_CODE"
    else:
      res = bm.verify(impl=code)
      return res.name

def run_single_best_k(k, max_examples=100, template=PromptPALM):
  def run_single (bm, type):
    # bm.build_prompt might do some heavy work like synthesizing examples with bosy
    prompt, examples = build_prompt(bm, params=bm.generate_params, template=template, mode=type, max_examples=max_examples)
    print(prompt, examples)
    best = "NO_CODE"
    for i in range(0, k):
      chat = chat_model.start_chat(examples=examples)
      response = chat.send_message(prompt, **parameters)
      code = extract_normalized_verilog_code(response.text, bm.name)
      if code == None:
        continue

      res = bm.verify(impl=code)
      if res ==  verify.ReturnCode.SUCCESS:
        return res.name
      elif res == verify.ReturnCode.FALSE_RESULT:
        best = res.name
      elif best != "FALSE_RESULT":
        best = res.name
    return best
  return run_single

def run_single_template(template=DefaultPromptTemplate):
  def run_single(bm, type):
    prompt = build_prompt(bm, params=bm.generate_params, template=template, mode=type)
    chat = chat_model.start_chat()
    response = chat.send_message(prompt, **parameters)
    print(response.text)
    code = extract_normalized_verilog_code(response.text, bm.name)
    if code == None:
      print(response.text)
      return "NO_CODE"
    else:
      res = bm.verify(impl=code)
      return res.name
  return run_single


def run_single_codechat(bm, type, template=DefaultPromptTemplate):
  print("Starting benchmark " + bm.name + "/" + type)
  # bm.build_prompt might do some heavy work like synthesizing examples with bosy
  prompt = build_prompt(bm, params=bm.generate_params, template=template, mode=type)
  chat = code_model.start_chat()
  response = chat.send_message(prompt, **parameters)
  code = extract_normalized_verilog_code(response.text, bm.name)
  if code == None:
    return "NO_CODE"
  else:
    res = bm.verify(impl=code)
    return res.name
    
def run_single_openai(bm, type):
  print("Starting benchmark " + bm.name + "/" + type)
  messages = build_prompt(bm, params=bm.generate_params, template=PromptOpenAI, mode=type)
  print(messages)
  
  try:
    completion = openai_complete(messages=messages)
  except openai.error.RateLimitError:
    return "AI_RATELIMIT"
  
  response = completion.choices[0].message.content
  print(response)
  code = extract_normalized_verilog_code(response, bm.name)
  if code == None:
    return "NO_CODE"
  else:
    res = bm.verify(impl=code, overwrite_params=bm.generate_params)
    return res.name

def run_single_openai_best_k(k, max_examples=100, model = "gpt-3.5-turbo-16k", template=PromptOpenAI):
  def run_single (bm, type):
    print("Starting benchmark " + bm.name + "/" + type)
    messages = build_prompt(bm, params=bm.generate_params, template=template, mode=type, max_examples=max_examples)
    
    try:
      completion = openai_complete(messages=messages, n = k, model=model)
    except openai.error.RateLimitError:
      return "AI_RATELIMIT"
    
    best = "NO_CODE"
    print(completion)
    for choice in completion.choices:
      response = choice.message.content
      code = extract_normalized_verilog_code(response, bm.name)
      if code == None:
        continue
      res = bm.verify(impl=code)
      if res ==  verify.ReturnCode.SUCCESS:
        return res.name
      elif res == verify.ReturnCode.FALSE_RESULT:
        best = res.name
      elif best != "FALSE_RESULT":
        best = res.name
    return best
  return run_single

def find_best_openai(bm : Benchmark, type : str):
    INCLUDE_SPECS = True
    parname, curval = list(bm.generate_params.items())[0]
    
    messages = build_prompt(bm, mode=type,template=PromptOpenAI, max_examples=1)
    print("Starting benchmark: " + bm.name)

    bestval = 0
    params = {parname : curval}
    while bestval < 1000:
      try:
        print("Current val = " + str(curval))
        completion = openai_complete(messages=messages, n=5, retry_n=40)

        res = -1
        for choice in completion.choices:
          response = choice.message.content
          code = extract_normalized_verilog_code(response, bm.name)
          #print(code)
          if code == None:
            continue
          res = bm.verify(impl=code, overwrite_params=params)
          if res ==  verify.ReturnCode.SUCCESS:
            break
        
        if res !=  verify.ReturnCode.SUCCESS: # we got wrong code :(
          return str(bestval)
        bestval = curval
        curval = curval * 2

        params = {parname : curval}
        messages.append(choice.message.to_dict())
        messages.append(
          build_prompt(bm, mode=type,template=PromptOpenAI if INCLUDE_SPECS else PromptOpenAINoSpecification, params=params)[-1]
        )


      except openai.error.RateLimitError: # OpenAI API seems to be quite unstable
        return "AI_RATELIMIT"
      except (openai.InvalidRequestError):
        messages = messages[0:1] + messages[2:1] # remove oldest message while keeping system instruction
    if code:
      print(code)
    return str(bestval)


In [11]:
async def run_benchmarks(benchmarks, file : str,run_single, example_types = ["self", "bosy", "strix", "none"], wait_time = 200 ):
  # setting up csv writing
  f = open(file, 'w', newline='')
  csvwriter = csv.DictWriter(f, fieldnames=["benchmark"] + example_types, dialect='unix', quoting=csv.QUOTE_NONE)
  csvwriter.writeheader()

  # get event loop to be able to run the requests in parallel
  loop = asyncio.get_event_loop()

  async def _run_single(bm, type):
    try:
      return await loop.run_in_executor(None, run_single, bm, type)
    except TimeoutError:
      return "TIMEOUT"
    except BosyError:
      return "BOSY_ERROR"
    except (google.api_core.exceptions.InternalServerError,
            google.api_core.exceptions.InvalidArgument,
          openai.InvalidRequestError) as e:
      print(e)
      return "AI_ERROR" 
    #except Exception as e:
    #  print(e, e.__class__)
  async def run_single_benchmark(bm, i = 0):
    result = {
      "benchmark": bm.name
    }
    await asyncio.sleep(wait_time * i)  # needed because of quota
    print(i)
    async_res = await asyncio.gather(
      *[_run_single(bm, type) for type in example_types]
    )
    for i, res in enumerate(async_res):
      result[example_types[i]] = res
    csvwriter.writerow(result)
    f.flush() # writes partial results to output

  await asyncio.gather(
    *[run_single_benchmark(bm, i) for i, bm in enumerate(benchmarks)]
  )
  f.close()

# await run_benchmarks(benchmarks=benchmarks_strix, run_single=run_single_openai_best_k(1), example_types=["bosy", "strix"], file = "results/strix_oneshot_gpt.csv", wait_time = 1000)
await run_benchmarks(benchmarks=benchmarks_bosy[0:2], run_single=run_single_best_k(3, max_examples=1, template=ModuleDefinitionPromptPALM), example_types=["none"], file = "results/_.csv", wait_time = 200)
#await run_benchmarks(benchmarks=benchmarks_strix, run_single=run_single_openai_best_k(k = 3, max_examples=2, model="gpt-4"), example_types=["strix", "none"], file = "results/strix_gpt4_best3.csv", wait_time = 600)
#await run_benchmarks(benchmarks=benchmarks_bosy, run_single=run_single_openai_best_k(k = 3, max_examples=2, model="gpt-4"), example_types=["bosy", "strix", "none"], file = "results/bosy_gpt4_best3.csv", wait_time = 600)

0
You are an expert in writing correct verilog code, that fulfill certain formal properties specified in LTL.
Please write a Verilog module for n=8 using the following module definition as a basis.
```verilog
module amba_decomposed_arbiter (input ALLREADY, input HBUSREQ_0, input HBUSREQ_1, input HBUSREQ_2, input HBUSREQ_3, input HBUSREQ_4, input HBUSREQ_5, input HBUSREQ_6, input HBUSREQ_7, output reg DECIDE, output reg BUSREQ, output reg HGRANT_0, output reg HGRANT_1, output reg HGRANT_2, output reg HGRANT_3, output reg HGRANT_4, output reg HGRANT_5, output reg HGRANT_6, output reg HGRANT_7)
```
The code needs to be fully synthesizable and follow the following LTL specification:
G (F ALLREADY) && ALLREADY -> G ((! HGRANT_0 && ! HGRANT_1 && ! HGRANT_2 && ! HGRANT_3 && (! HGRANT_4 && ! HGRANT_5 && (! HGRANT_6 && true || true && (! HGRANT_7)) || (! HGRANT_4 && true || true && (! HGRANT_5)) && (! HGRANT_6 && ! HGRANT_7)) || (! HGRANT_0 && ! HGRANT_1 && (! HGRANT_2 && true || true && (! HGR

In [15]:
#dot = bosy.synthesize(read_file("detector.tlsf"), target="dot", overwrite_params={"n":2})
#print(dot)
#ltl = syfco.convert(read_file("detector.tlsf"), format="ltl", overwrite_params={"n": 4})
#print(ltl)
benchmarks[0].verify("""module detector(
  input [1:0] r,
  input clk,
  output reg g
);
  assign g = 0;
endmodule
""", overwrite_params={"n": 2})

Converting Verilog to AIGER

 /----------------------------------------------------------------------------\
 |                                                                            |
 |  yosys -- Yosys Open SYnthesis Suite                                       |
 |                                                                            |
 |  Copyright (C) 2012 - 2020  Claire Xenia Wolf <claire@yosyshq.com>         |
 |                                                                            |
 |  Permission to use, copy, modify, and/or distribute this software for any  |
 |  purpose with or without fee is hereby granted, provided that the above    |
 |  copyright notice and this permission notice appear in all copies.         |
 |                                                                            |
 |  THE SOFTWARE IS PROVIDED "AS IS" AND THE AUTHOR DISCLAIMS ALL WARRANTIES  |
 |  WITH REGARD TO THIS SOFTWARE INCLUDING ALL IMPLIED WARRANTIES OF          |
 |  MERCHAN

<ReturnCode.FALSE_RESULT: 11>

In [ ]:

contents = read_file("detector.tlsf")
ltl = syfco.convert(contents, "ltl", overwrite_params={"n": 4})
bosy_f = syfco.convert(contents, "bosy", overwrite_params={"n": 4})
bosy_impl = bosy.synthesize(bosy_f)

template = PromptTemplate()
template.add_example({
    "SPEC": syfco.convert(contents, "ltl", overwrite_params={"n": 2}),
    "IMPL": read_file("../../verilog/detector/detector_2.vl"),
    "PARAMS": "n=2"
})
prompt = template.build_prompt({"SPEC": syfco.convert(contents, "ltl", overwrite_params={"n": 4})})
print(prompt)

You are an expert in writing correct verilog code, that fulfill certain formal properties specified in LTL.
Here is an example for n=2. It satisfies the LTL specification G (F r_0) && G (F r_1) <-> G (F g):
module detector(
  input [1:0] r,
  input clk,
  output reg g
);
  reg [1:0] state;
  initial state = '0;
  always @(posedge clk) begin
    state = state | r;
    g = 0;
    if(state == '1) begin
      g = 1;
      state = '0;
    end
  end
endmodule

Please write a Verilog module fulfilling the following expectations. Make sure the code is fully synthesizable.:
G (F r_0) && G (F r_1) && G (F r_2) && G (F r_3) <-> G (F g)
